In [9]:
from bs4 import BeautifulSoup
import requests as req
import numpy as np
import pandas as pd

In [10]:
def parse(source: str, source_link=False) -> pd.DataFrame:

    # with open('test_parser.html') as fp:
      # soup = BeautifulSoup(fp, 'html.parser')

    if source_link:

        web = req.get(source)
        soup = BeautifulSoup(web.text, 'html.parser')

    else:
    
        soup = BeautifulSoup(source)

    df = pd.DataFrame([], columns=['name', 'price', 'old_price', 'sizes_available'])

    for i, el in enumerate(soup.findAll('div', class_='offer_item')):

        name = el.find('div', class_='name').a.text
        price = el.find('div', class_='price')

        if el.find('div', class_='sizes_available') is not None:
            sizes = [x.text for x in el.find('div', class_='sizes_available').find('div').findAll('a')]
            for j, size in enumerate(sizes):
                sizes[j] = ''.join([x for x in size if x.isalnum()])
        else:
            sizes = np.NaN

        if price.find('span', class_='old-price') is not None:

            spec_price = price.find('span', class_='spec-price').text
            old_price = price.find('span', class_='old-price').text

            df.loc[i] = [name,
                        float(''.join([x for x in spec_price if x.isdigit()])),
                        float(''.join([x for x in old_price if x.isdigit()])),
                        sizes]
        else:
            price_ = price.find('span').text
            df.loc[i] = [name,
                        int(''.join([x for x in price_ if x.isdigit()])),
                        np.NaN,
                        sizes]

    return df

In [11]:
# link = 'https://21-shop.ru/catalog/muzhskoe/odezhda/bryuki/?PAGEN_1=3&more=1'
link = 'https://21-shop.ru/catalog/muzhskoe/aksessuary/ochki/'
df = parse(link, source_link=True)
df

,name,price,old_price,sizes_available
0,Очки IZIPIZI SLMSIC33_00 Черепаховые зеркальные,6700,NaN,NaN
1,Очки IZIPIZI SLMSCC103_00 Черепаховые зеленые ...,6700,NaN,NaN
2,Очки IZIPIZI SLMSDC145_00 Розовый кварц,6700,NaN,NaN
3,Очки TRUESPIN Petrichor Black Fade,990,NaN,NaN
4,Очки TRUESPIN Candor Black,990,NaN,NaN
5,Очки IZIPIZI NAUTIC NAUTICPAC46_00 Голубо-чере...,7125,NaN,NaN
6,Очки TRUESPIN Candor Art Black,990,NaN,NaN
7,Очки TRUESPIN Candor Art White,990,NaN,NaN


In [11]:
!pip install dominate


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import unittest
from dominate import document
from dominate.tags import *
import numpy as np
from pandas.testing import *

In [13]:
def generate_html(df: pd.DataFrame):


  with document(title='Shop') as doc:
    for index, row in df.iterrows():

      name = row['name']
      old_price = row['old_price']
      spec_price = row['price']
      sizes_available = row['sizes_available']

      with div(_class='offer_item'):
        with div(_class='name'):
          a(name)
        with div(_class='price'):
          if pd.isna(old_price):
            span(spec_price)
          else:
            span(old_price, _class='old-price')
            span(spec_price, _class='spec-price')
        if sizes_available is not None:
          with div(_class='sizes_available'):
            with div():
              for i in sizes_available:
                a(i)
  return doc.render()

class TestParser(unittest.TestCase):
  def test_parser(self):

    test_df = pd.DataFrame([
      ['Брюки', 1111,2222, ['XX', 'SX']],
      ['Юбки', 3333, None, ['XX', 'SX', 'S', 'M']],
      ['Очки', 4444, None, None]
    ], columns=['name', 'price', 'old_price', 'sizes_available'], dtype=object)
    expected = test_df.replace({np.nan: None})

    html_page = generate_html(test_df)
    parsed = parse(html_page)

    # assert_frame_equal(parsed, expected, check_dtype=False)
    assert_frame_equal(parsed, expected)

unittest.main(argv=[''], verbosity=2, exit=False)

test_parser (__main__.TestParser.test_parser) ... FAIL

FAIL: test_parser (__main__.TestParser.test_parser)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Ilya\AppData\Local\Temp\ipykernel_23716\181152910.py", line 42, in test_parser
    assert_frame_equal(parsed, expected)
  File "d:\Program Files\Python\Python311\Lib\site-packages\pandas\_testing\asserters.py", line 1209, in assert_frame_equal
    assert_series_equal(
  File "d:\Program Files\Python\Python311\Lib\site-packages\pandas\_testing\asserters.py", line 918, in assert_series_equal
    assert_attr_equal("dtype", left, right, obj=f"Attributes of {obj}")
  File "d:\Program Files\Python\Python311\Lib\site-packages\pandas\_testing\asserters.py", line 414, in assert_attr_equal
    raise_assert_detail(obj, msg, left_attr, right_attr)
  File "d:\Program Files\Python\Python311\Lib\site-packages\pandas\_testing\asserters.py", line 598, in raise_assert_detail
 

In [14]:
result = parse(html_page)
result

NameError: name 'html_page' is not defined